In [1]:
import sys
sys.path.append('../')
from data.dataset import *
from evaluate import *
import tensorflow as tf
import tensorflow.keras.layers as tfkl
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np
import os
import time

2021-08-16 20:55:04.044258: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Cargo el dataset de texto y le aplico las transformaciones para poder usarlo para entrenar

In [2]:
path_to_file = '../data/chamame-web/txts/chamame_dataset.txt'

encoded_data, max_id, tokenizer = dataset_pre(path_to_file)
train_set, val_set = dataset_split(encoded_data, train_s = 90)
data_train = window_sequence(train_set, 50, 32, max_id, 1)
data_val = window_sequence(val_set, 100, 32, max_id, 512)

2021-08-16 20:55:05.880958: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-16 20:55:05.926013: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-16 20:55:05.926239: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.7715GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2021-08-16 20:55:05.926255: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-16 20:55:05.928952: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-08-16 20:55:05.928976: I tensorflow/stream_executo

Defino el modelo stateless. El recurrent drop out lo dejo en cero para evitar el warning de cuda

In [3]:
tf.keras.backend.clear_session()

entrada = tfkl.Input((None, max_id), name='ENTRADA')

capa_rec = tfkl.GRU(128, return_sequences=True, input_shape=[None, max_id],dropout=0.2, recurrent_dropout=0)(entrada)
capa_rec = tfkl.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0)(capa_rec)

salida = tfkl.TimeDistributed(tfkl.Dense(max_id, activation="softmax"))(capa_rec)

model = tf.keras.Model(inputs=[entrada], outputs=[salida])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")


Entrenamiento

In [4]:
history = model.fit(data_train, epochs=50)

2021-08-16 20:55:10.992008: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-16 20:55:11.009904: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3199980000 Hz


Epoch 1/50


2021-08-16 20:55:14.212083: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-08-16 20:55:14.474914: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8202
2021-08-16 20:55:14.573926: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


     11/Unknown - 4s 6ms/step - loss: 3.6264

2021-08-16 20:55:14.761941: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


1248/1248 [==============================] - 15s 9ms/step - loss: 2.1515
Epoch 2/50
1248/1248 [==============================] - 13s 9ms/step - loss: 1.7881
Epoch 3/50
1248/1248 [==============================] - 12s 8ms/step - loss: 1.6527
Epoch 4/50
1248/1248 [==============================] - 13s 9ms/step - loss: 1.5622
Epoch 5/50
1248/1248 [==============================] - 12s 8ms/step - loss: 1.4996
Epoch 6/50
1248/1248 [==============================] - 14s 10ms/step - loss: 1.4491
Epoch 7/50
1248/1248 [==============================] - 14s 10ms/step - loss: 1.4093
Epoch 8/50
1248/1248 [==============================] - 14s 10ms/step - loss: 1.3772
Epoch 9/50
1248/1248 [==============================] - 12s 8ms/step - loss: 1.3520
Epoch 10/50
1248/1248 [==============================] - 13s 9ms/step - loss: 1.3273
Epoch 11/50
1248/1248 [==============================] - 13s 9ms/step - loss: 1.3081
Epoch 12/50
1248/1248 [==============================] - 14s 10ms/step - loss: 1.2

In [8]:
model.save_weights('pesos.hdf5')

Hago predicciones a partir del modelo entrenado

In [9]:
model.load_weights('pesos.hdf5')

In [19]:
print(complete_text(model, "a", tokenizer, n_chars=200, temperature=0.45))

a su para del caraná. 


tomador de mi provincia 
porá la luna, candor de niña. 


para morir, 
y apagar por esta aquel alega 
por un vino la losa añoso, bosquejo en las ojos 
canoó de mi provincia 
po
